In [23]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.backend as K
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import logging
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import sys
import mysql.connector

HOST="143.244.188.157"
PORT="3306"
USER="patrick-finProj"
PASSWORD="Pat#21$rick"

try: 
    conn = mysql.connector.connect(
        host=HOST,
        port=PORT,
        user=USER,
        password=PASSWORD,
        database="GlobalMarketData"
    )
    query = f"SELECT * FROM histdailyprice4;"
    histdailyprice4 = pd.read_sql(query, conn)
    conn.close()
except Exception as e:
    conn.close()
    print(str(e))

In [12]:
df = histdailyprice4.copy()
df.Date = pd.to_datetime(df.Date)
df.set_index('Date', inplace=True)
df.head()

,Symbol,Exchange,Close,Open,High,Low,Volume,AdjClose,ActualPercent
Date,,,,,,,,,
2000-01-03,AAPL,NASDAQ,111.94,104.87,112.50,101.69,4783900,0.855709,8.433090
2000-01-03,AMAT,NASDAQ,126.50,128.40,128.40,125.30,3589100,23.712400,4.901190
2000-01-03,AMD,NASDAQ,31.00,29.94,31.19,29.38,3921600,15.500000,5.645160
2000-01-03,BA,NYSE,40.19,41.63,41.69,39.81,2638200,25.897600,0.149291
2000-01-03,BAC,NYSE,48.44,50.25,50.25,48.00,6852900,13.043600,5.945500


In [36]:
train_cols = ["Close", "Open", "High", "Low", "Volume", "AdjClose", "ActualPercent"]

# Split data into training set and test set
df_train, df_test = train_test_split(df, test_size=0.1, shuffle=False)

# Scale feature MinMax
min_max_scaler = MinMaxScaler()
X_train = min_max_scaler.fit_transform(df_train.loc[:,train_cols].values)
X_test = min_max_scaler.transform(df_test.loc[:,train_cols])

In [37]:
# Set model parameters
params = {
    "BATCH_SIZE": 50,
    "EPOCHS": 50,
    "LR": 0.00010000,
    "TIME_STEPS": 60
    }

TIME_STEPS = params['TIME_STEPS']
BATCH_SIZE = params['BATCH_SIZE']

In [38]:
def trim_dataset(mat, batch_size):
    """Trims dataset to a size that's divisible by BATCH_SIZE"""
    
    no_of_rows_drop = mat.shape[0] % batch_size

    if no_of_rows_drop > 0:
        return mat[:-no_of_rows_drop]
    else:
        return mat

def build_timeseries(mat, y_col_index):
    """Transform data into time series format"""
    
    dim_0 = mat.shape[0] - TIME_STEPS
    dim_1 = mat.shape[1]

    X = np.zeros((dim_0, TIME_STEPS, dim_1))
    y = np.zeros((dim_0,))

    print("Length of inputs", dim_0)

    for i in range(dim_0):
        X[i] = mat[i:TIME_STEPS+i]
        y[i] = mat[TIME_STEPS+i, y_col_index]

    print("length of time-series - inputs", X.shape)
    print("length of time-series - outputs", y.shape)

    return X, y

X_t, y_t = build_timeseries(X_train, 3)
X_t = trim_dataset(X_t, BATCH_SIZE)
y_t = trim_dataset(y_t, BATCH_SIZE)
print("Batch trimmed size", X_t.shape, y_t.shape)

Length of inputs 412596
length of time-series - inputs (412596, 60, 7)
length of time-series - outputs (412596,)
Batch trimmed size (412550, 60, 7) (412550,)


In [39]:
def custom_loss(y_true, y_pred):
    """Customized loss function that takes into account directional loss.
    
    ARGS:
    y_true: tensor of true price
    y_pred: tensor of predicted price
    
    RETURN:
    custom loss output
    """
    try:
        #the "next day's price" of tensor
        y_true_next = y_true[1:]
        y_pred_next = y_pred[1:]

        #the "today's price" of tensor
        y_true_tdy = y_true[:-1]
        y_pred_tdy = y_pred[:-1]

        #substract to get up/down movement of the two tensors
        y_true_diff = tf.subtract(y_true_next, y_true_tdy)
        y_pred_diff = tf.subtract(y_pred_next, y_pred_tdy)

        #create a standard tensor with zero value for comparison
        standard = tf.zeros_like(y_pred_diff)

        #compare with the standard; if true, UP; else DOWN
        y_true_move = tf.greater_equal(y_true_diff, standard)
        y_pred_move = tf.greater_equal(y_pred_diff, standard)
        y_true_move = tf.reshape(y_true_move, [-1])
        y_pred_move = tf.reshape(y_pred_move, [-1])


        #find indices where the directions are not the same
        condition = tf.not_equal(y_true_move, y_pred_move)
        indices = tf.where(condition)

        #move one position later
        ones = tf.ones_like(indices)
        indices = tf.add(indices, ones)
        indices = K.cast(indices, dtype='int32')


        #create a tensor to store directional loss and put it into custom loss output
        direction_loss = tf.Variable(tf.ones_like(y_pred), dtype='float32')
        updates = K.cast(tf.ones_like(indices), dtype='float32')
        alpha = 1000
        direction_loss = tf.scatter_nd_update(direction_loss, indices, alpha*updates)

        custom_loss = K.mean(tf.multiply(K.square(y_true - y_pred), direction_loss), axis=-1)

        return custom_loss
    except Exception as e:
        logging.error("Exception occurred at get_price_movement()", exc_info=True)

In [40]:
def create_lstm_model():
    """Build LSTM model"""
      
    lstm_model = Sequential()
    lstm_model.add(LSTM(100, batch_input_shape=(BATCH_SIZE, TIME_STEPS, X_t.shape[2]),
                        dropout=0.0, recurrent_dropout=0.0,
                        stateful=True, return_sequences=True,
                        kernel_initializer='random_uniform'))

    lstm_model.add(LSTM(60, dropout=0.0))
    
    lstm_model.add(Dense(20,activation='relu'))
    lstm_model.add(Dense(1, activation='sigmoid'))
        
    #Compile the model
    optimizer = optimizers.Adam(lr=params["LR"])
    lstm_model.compile(loss=custom_loss, optimizer=optimizer)
  
    return lstm_model

In [42]:
'''Split test data into validation set and test set'''
X_temp, y_temp = build_timeseries(X_test, 3)
X_val, X_test_t = np.array_split(trim_dataset(X_temp, BATCH_SIZE), 2)
y_val, y_test_t = np.array_split(trim_dataset(y_temp, BATCH_SIZE), 2)
print("Test size", X_test_t.shape, y_test_t.shape, X_val.shape, y_val.shape)

Length of inputs 45791
length of time-series - inputs (45791, 60, 7)
length of time-series - outputs (45791,)
Test size (22875, 60, 7) (22875,) (22875, 60, 7) (22875,)
